In [1]:
from kotodama_model2 import AutoEncoder
from kotodama_data import ToData
import config
import torch
from torch.utils import data as DataUtil
import h5py
import pytorch_lightning as pl

In [2]:
class OriginalData(DataUtil.Dataset):
    def __init__(self) -> None:
        super().__init__()

        with h5py.File(ToData.file_name,'r') as f:
            self.acurrent = torch.from_numpy(f[ToData.current_key][...])
            self.memory = torch.from_numpy(f[ToData.memory_key][...])
        print(self.acurrent.shape,self.memory.shape)

    def __len__(self):
        return self.acurrent.size(0)
    
    def __getitem__(self, index):
        return self.acurrent[index],self.memory[index]
data_set = OriginalData()

torch.Size([97, 129, 128]) torch.Size([97, 8, 16, 128])


In [3]:
model = AutoEncoder()
batch_size = 32
EPOCHS = 1000
data_loader = DataUtil.DataLoader(data_set,batch_size,shuffle=True)

In [4]:
trainer = pl.Trainer(gpus=1,num_nodes=1,precision=16,max_epochs=EPOCHS)
trainer.fit(model,data_loader)
from datetime import datetime
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S-%f')
name = f'params/KotodamaAutoEncoder2_{now}.params'
torch.save(model.state_dict(),name)
print('saved')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | criterion | MSELoss | 0     
1 | encoder   | Encoder | 1.8 M 
2 | decoder   | Decoder | 2.7 M 
--------------------------------------
4.5 M     Trainable params
0         Non-trainable params
4.5 M     Total params
18.018    Total estimated model params size (MB)


Epoch 0:   0%|          | 0/4 [00:00<?, ?it/s] 

C:\Users\22shi\anaconda3\envs\JARVIS\lib\site-packages\pytorch_lightning\trainer\data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 9: 100%|██████████| 4/4 [00:00<00:00,  4.84it/s, loss=0.281, v_num=2]
saved
